In [83]:
#coding:utf-8
import nltk

In [84]:
# nltk.download()

In [85]:
# The code was removed by DSX for sharing.

In [86]:
#预处理
#根据分词结果调整jieba
add_list = np.array(pd.read_csv(get_object_storage_file('MyHackathon', 'add_list.csv')))

for item in add_list:
    jieba.add_word(item[0],tag=item[1])

# 去除停用词并筛选出名词
clean_words=[]
ps = PorterStemmer()
for line in question:
    seg = jieba.posseg.cut(line[0])
    for j in seg:
        if j.flag!='eng'and j.word not in stop_words and j.flag == 'n':   
            clean_words.append(j.word)
        elif j.flag=='eng' and j.word not in stopwords.words('english') :
            eng_tag=nltk.pos_tag(nltk.word_tokenize(j.word))
            for w,pos in eng_tag:
                if pos.startswith('NN'):
                    clean_words.append(w)    
corpus=[]                   
for line in question:
    #分词
    seg_generator = jieba.cut(line[0])
    #筛出stop word
    seg_list = [i for i in seg_generator if i in clean_words]
    seg_list = r' '.join(seg_list)
    corpus.append(seg_list.strip())
print ("预处理后：",corpus)

预处理后： ['Agile Office', 'CDL 办公室管理 挑战 工作日 小朋友 家长 学校 小朋友 成年人 家长 问题 实验室 工作环境 性质 security 要求 小朋友 小朋友 同事 工作 小朋友 办公设备 情况 投影幕布 马克笔 小朋友 办公区 活动 安全隐患 问题 看法 建议', '工程师 日常生活 改变 e g 工作态度 社交活动 生活习惯 创新思考', 'Dear IBMer IBM 业界 巅峰 mind help IBM top company industry', '项目 学习 技术 时间', 'IBM 转型 成功 技术 客户需求', '工程师文化 前 词 自由 效率 平等 尊重 信任 创新 流程 解决问题 导向', 'Whitewater 工具', '工作环境 改变', 'IBM 技术 导向', '产品 项目 人才 成功 市场 产品经理 技术 开发 测试 运维 销售', 'Lab 工作 心情', 'IT 产业 变革 IBM 转型 IBMer 产品 项目 努力 转变 产品 成功 CDL 转型 成功 IBM 转型 成功', '工作 转变 Transformation 职业规划 转变 知识技能体系 工作习惯 沟通管理方式 方面 转变', '技术 技术', 'love IBM', 'ideas team building Sep enjoy BJ autumn', '团队氛围 激情 内心的声音 团队成员 动能 感觉 团队 团队 山 野炊 篮球 足球 OK 技术难题 客户 领导 公司预算 活动 需求 AA 钱 事', 'IBM 原因', 'API 问题 活动 culture transformation 感觉', '开源软件 企业 改变', '工作模式', 'David Kenny TownHall Powerpoint', '功能 表单 submit 单选框 文字 etc', 'work hate', 'OSPF 系统 改进', 'Pls recommend technology skill development approach', 'Agile workspace 有味', '工作 技术', 'Jam', '机会 技术 开发', '程序 生命 重构 代码', '时 小事', '公司 管理层 BUE CDL CDL 外', '开会厌烦症 原

In [87]:
#词频矩阵
stop_words=[]
stop_list=np.array(pd.read_csv(get_object_storage_file('MyHackathon', 'stop_list.csv')))
for item in stop_list:
    stop_words.append(item[0])
    
vectorizer = CountVectorizer(stop_words=stop_words)
#fit_transform是将文本转为词频矩阵
X = vectorizer.fit_transform(corpus)
#获取词袋模型中的所有词语
word = vectorizer.get_feature_names()
print ("所有词语",word)
weight = X.toarray()
print("词频矩阵",weight)

所有词语 ['aa', 'activity', 'agile', 'ai', 'api', 'approach', 'bj', 'bu', 'bue', 'bugzilla', 'cdl', 'community', 'company', 'culture', 'defect', 'development', 'disclosure', 'environment', 'fc', 'fear', 'git', 'gvt', 'hibernate', 'ibm', 'ibmer', 'ideas', 'industry', 'invention', 'it', 'jam', 'jquery', 'lab', 'learn', 'meeting', 'mind', 'money', 'office', 'openwhisk', 'organization', 'ospf', 'person', 'plan', 'pm', 'powerpoint', 'project', 'rtc', 'security', 'sessions', 'skill', 'smash', 'spring', 'study', 'team', 'technology', 'tomcat', 'tool', 'topic', 'townhall', 'transformation', 'tvt', 'whitewater', 'work', 'workspace', 'zero', '下午茶', '不足', '业界', '个人时间', '云计算技术', '产业', '产品', '产品经理', '人才', '代码', '企业', '优势', '信任', '充满热情', '公司', '公司预算', '内心的声音', '创新', '创新思考', '办公区', '办公室管理', '办公设备', '办法', '功能', '动能', '努力', '单选框', '原因', '变革', '同事', '团队', '团队成员', '团队氛围', '士气', '外界', '学习', '学校', '安全性', '安全隐患', '实验室', '客户', '客户需求', '家长', '导向', '尊重', '小事', '小孩', '小朋友', '小确幸', '巅峰', '工作', '工作习惯', '工作态度', '工作日',

In [88]:
#建立模型,10个主题,迭代1500次,超参数αβ分别为50/T+1、200/W
topic_num=10
word_num=len(word)
model=lda.LDA(n_topics=topic_num, n_iter=1500, random_state=1,alpha=(50.0/topic_num)+1,eta=0.001)
X=np.asarray(weight)
model.fit(X)

INFO:lda:n_documents: 84
INFO:lda:vocab_size: 200
INFO:lda:n_words: 328
INFO:lda:n_topics: 10
INFO:lda:n_iter: 1500
INFO:lda:<0> log likelihood: -3623
INFO:lda:<10> log likelihood: -2845
INFO:lda:<20> log likelihood: -2831
INFO:lda:<30> log likelihood: -2836
INFO:lda:<40> log likelihood: -2829
INFO:lda:<50> log likelihood: -2849
INFO:lda:<60> log likelihood: -2833
INFO:lda:<70> log likelihood: -2829
INFO:lda:<80> log likelihood: -2830
INFO:lda:<90> log likelihood: -2827
INFO:lda:<100> log likelihood: -2827
INFO:lda:<110> log likelihood: -2834
INFO:lda:<120> log likelihood: -2853
INFO:lda:<130> log likelihood: -2828
INFO:lda:<140> log likelihood: -2834
INFO:lda:<150> log likelihood: -2827
INFO:lda:<160> log likelihood: -2826
INFO:lda:<170> log likelihood: -2828
INFO:lda:<180> log likelihood: -2836
INFO:lda:<190> log likelihood: -2825
INFO:lda:<200> log likelihood: -2834
INFO:lda:<210> log likelihood: -2831
INFO:lda:<220> log likelihood: -2820
INFO:lda:<230> log likelihood: -2832
INFO:ld

In [91]:
topic_word = model.topic_word_ 
n = 3
# print("主题-单词分布矩阵:")
# for index , value in enumerate(topic_word):
#     print(value)
#输出单词主题中的TopN关键词
word = vectorizer.get_feature_names()

topic_words_list = []
for i, topic_dist in enumerate(topic_word):
    print (max(topic_dist))
#     topic_words = np.array(word)[np.argsort(topic_dist)][:-(n+1):-1]
#     topic_words_list.append(' '.join(topic_words))
    
# print("主题-单词分布矩阵,取confidence的前2个单词:")
# for index , value in enumerate(topic_words_list):
#     print("topic_words%d: "%index, value)

0.0971116504854
0.268844086022
0.198708609272
0.369346590909
0.137020547945
0.231821192053
0.21280141844
0.165773480663
0.331519337017
0.114541984733


In [94]:
#文档-主题分布
doc_topic = model.doc_topic_
# print("文档-主题分布矩阵:")
# for index , value in enumerate(doc_topic):
#     print(value)

doc_topic_list = []

print("文档-主题分布对应的词主题分布:")
topic_num=2
for i in range(len(doc_topic)):
    #文档-主题分布中最大的词主题分布：每篇文档对应一个主题
    doc_topics = np.array(word)[np.argsort(topic_dist)][:-(topic_num+1):-1]    
    doc_topic_list.append(' '.join(doc_topics))

print (doc_topic_list)
    
#     max_topic = doc_topic[i].argmax()
#     print (doc_topic[i])
#     print("doc_topic%d: "%i, "   topic_word %d"%max_topic)
#     doc_topic_list.append(topic_words_list[max_topic])
# print("文档-主题分布矩阵,拟合主题词矩阵后:")
# for index , value in enumerate(doc_topic_list):
# #      print("doc_topic%d: "%index, value)  
#      print(value)   

文档-主题分布对应的词主题分布:
['马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导', '马克笔 领导']
